In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import random
import sys, os.path, os
from tqdm import tqdm
import pickle as pkl
from shapely.geometry import Point,Polygon
import dateutil.parser

import random
from datetime import datetime, timedelta

import sys, os.path, os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    path_opencp="drive/Shareddrives/FAIRNESS/Colab/Librerias/PredictCode/"
    path_fairness="drive/Shareddrives/FAIRNESS/Colab/Librerias/"
    path_exp="drive/Shareddrives/FAIRNESS/Colab/Notebooks_for_topic/EXP0/SCRIPTS/"

except:

    path_opencp="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Librerias\\PredictCode"
    path_fairness="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Librerias"
    path_exp="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Notebooks_for_topic\\EXP0\\SCRIPTS"

sys.path.insert(0, os.path.abspath(path_opencp))
sys.path.insert(0, os.path.abspath(path_fairness))
sys.path.insert(0, os.path.abspath(path_exp))

import open_cp
import open_cp.geometry
import matplotlib.pyplot as plt

try:
    os.chdir("drive/Shareddrives/FAIRNESS/Colab/Notebooks_for_topic/EXP0/DATOS/CHICAGO")
except:
    os.chdir("g:\\Unidades compartidas\\FAIRNESS\\Colab\\Notebooks_for_topic\\EXP0\\DATOS\\CHICAGO")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ERROR:open_cp.geometry:Failed to import `rtree`.


In [ ]:
from models.model_selection import NAIVE_MODEL, KDE_MODEL, SEPP_MODEL
from open_cp.sources.sepp import make_time_unit
from global_vars import days_time_unit
import datetime
from datetime import timedelta

def intensity_m(model,date,region,grid_size):

    try:
        predict=model.predict(date)
    except:
        predict=model.predict()

    try:
        M=open_cp.predictors.GridPredictionArray.from_continuous_prediction_region(predict, region, grid_size, grid_size)
        intensity_matrix= M.intensity_matrix
    except:
        intensity_matrix = predict.intensity_matrix

    if intensity_matrix.sum()==0:
        return intensity_matrix

    else:
        return intensity_matrix/intensity_matrix.sum()

### datos reales
import open_cp.naive as naive
def get_real_m(timedpoints,p_date,grid_size,region):
    try:
        real = timedpoints[(timedpoints.times_datetime()>=p_date) & ((timedpoints.times_datetime()<p_date+timedelta(days=1)))]
    except:
        p_date=datetime.datetime.combine(pred_date,datetime.time())
        real = timedpoints[(timedpoints.times_datetime()>=p_date) & ((timedpoints.times_datetime()<p_date+timedelta(days=1)))]

    predictor = naive.CountingGridKernel(grid_size,grid_size,region=region)
    predictor.data = real
    try:
        gridpred = predictor.predict().renormalise()
        real=gridpred.intensity_matrix.data

        return real
    except:
        return predictor.predict().intensity_matrix

Failed to load 'descartes' package.
ERROR:open_cp.network:Failed to import `rtree`.


## <center> <font color='red'>7. Train and predict Models

In [ ]:
Train_chicago = pkl.load(open("Train_chicago.pkl", "rb")) ## cargar datos
Test_chicago = pkl.load(open("Test_chicago.pkl", "rb")) ## cargar datos
region = pkl.load(open("chicago_definition_region.pkl", "rb")) ## cargar datos

In [ ]:
import open_cp.seppexp as sepp

In [ ]:
### SEPP
path_file=os.path.join("SEPP/modelgrid.pkl")

if not os.path.exists(path_file):
    trainer = sepp.SEPPTrainer(region, grid_size=1000)
    trainer.data = Train_chicago
    sepp_model_result = trainer.train(iterations=50)
    sepp_model_result.data = Train_chicago
    pkl.dump(sepp_model_result, open(path_file, "wb"))

else:
    sepp_model_result=pkl.load(open(path_file, "rb"))

In [ ]:
if not os.path.exists(os.path.join("SEPP/","predictions_grid.pkl")):
    predictions={}
else:
    predictions=pkl.load(open(os.path.join("SEPP/","predictions_grid.pkl"), "rb"))

if not os.path.exists(os.path.join("SEPP/","reals_grid.pkl")):
    reals={}
else:
    reals=pkl.load(open(os.path.join("SEPP/","reals_grid.pkl"), "rb"))


for i in tqdm(range(150)):
    pred_date=Test_chicago.time_range[0].astype(datetime.datetime).date()+timedelta(days=i)
    if not pred_date in predictions:
        predict=intensity_m(sepp_model_result,pred_date,region,1000)
        predictions[pred_date]=predict

    # real data
    if not pred_date in reals:
        real=get_real_m(Test_chicago,pred_date,1000,region)
        reals[pred_date]=real
    # try:
    #     EMD[pred_date]=EMD(real,predict,25000000)
    # except:
    #     pass


    pkl.dump(predictions, open(os.path.join("SEPP/","predictions_grid.pkl"), "wb"))
    pkl.dump(reals, open(os.path.join("SEPP/","reals_grid.pkl"), "wb"))


100%|██████████| 150/150 [00:16<00:00,  8.98it/s]


In [2]:
predictions=pkl.load(open(os.path.join("SEPP/","predictions_grid.pkl"), "rb"))
predictions_naive=pkl.load(open(os.path.join("NAIVE/","predictions2023.pkl"), "rb"))
predictions_kde=pkl.load(open(os.path.join("KDE/","predictions2023.pkl"), "rb"))

In [3]:
len(predictions),len(predictions_naive),len(predictions_kde)

(150, 152, 151)

In [4]:
kde_list=[]
for i in predictions_kde:
  #print(i)
  kde_list.append(i)

naive_list=[]
for i in predictions_naive:
  #print(i)
  naive_list.append(i)

sepp_list=[]
for i in predictions:
  #print(i)
  sepp_list.append(i)

In [5]:
set(predictions_naive)-set(kde_list), set(kde_list)-set(predictions_naive)

({datetime.datetime(2023, 6, 1, 0, 0)}, set())

In [13]:
import datetime
predictions_kde[datetime.datetime(2023, 2, 7, 0, 0)]

array([[3.01981105e-56, 1.65919119e-52, 5.13602052e-48, ...,
        5.31369153e-05, 5.47229679e-05, 5.16832663e-05],
       [1.16398747e-53, 2.83675472e-50, 3.92417274e-46, ...,
        8.80442678e-05, 9.56879025e-05, 9.19144384e-05],
       [4.15059465e-51, 6.38762065e-48, 2.64992137e-44, ...,
        1.51577648e-04, 1.65988956e-04, 1.53107698e-04],
       ...,
       [7.38390081e-05, 1.37495197e-04, 1.43200816e-04, ...,
        1.55087639e-15, 4.34399265e-18, 6.48832978e-21],
       [6.72948403e-05, 1.05375748e-04, 9.23725878e-05, ...,
        2.66941405e-16, 6.39588520e-19, 7.99945247e-22],
       [5.24903127e-05, 6.92585773e-05, 5.11686754e-05, ...,
        4.53889204e-17, 8.14211988e-20, 9.31758842e-23]])

In [17]:
for i in set(kde_list):
  print(i)

2023-04-25 00:00:00
2023-04-13 00:00:00
2023-02-23 00:00:00
2023-04-02 00:00:00
2023-05-09 00:00:00
2023-02-19 00:00:00
2023-02-20 00:00:00
2023-03-20 00:00:00
2023-03-07 00:00:00
2023-02-05 00:00:00
2023-02-02 00:00:00
2023-04-08 00:00:00
2023-05-18 00:00:00
2023-01-26 00:00:00
2023-05-27 00:00:00
2023-04-11 00:00:00
2023-05-03 00:00:00
2023-04-06 00:00:00
2023-01-27 00:00:00
2023-04-17 00:00:00
2023-05-20 00:00:00
2023-02-06 00:00:00
2023-03-22 00:00:00
2023-02-18 00:00:00
2023-02-25 00:00:00
2023-01-16 00:00:00
2023-01-21 00:00:00
2023-01-05 00:00:00
2023-04-29 00:00:00
2023-05-02 00:00:00
2023-04-27 00:00:00
2023-05-31 00:00:00
2023-05-16 00:00:00
2023-03-03 00:00:00
2023-05-26 00:00:00
2023-01-14 00:00:00
2023-01-09 00:00:00
2023-04-04 00:00:00
2023-03-23 00:00:00
2023-01-25 00:00:00
2023-05-25 00:00:00
2023-02-26 00:00:00
2023-05-22 00:00:00
2023-05-12 00:00:00
2023-05-01 00:00:00
2023-03-28 00:00:00
2023-02-12 00:00:00
2023-04-20 00:00:00
2023-01-01 00:00:00
2023-04-16 00:00:00


In [10]:
for i in predictions:
  print(i)

2023-01-01
2023-01-02
2023-01-03
2023-01-04
2023-01-05
2023-01-06
2023-01-07
2023-01-08
2023-01-09
2023-01-10
2023-01-11
2023-01-12
2023-01-13
2023-01-14
2023-01-15
2023-01-16
2023-01-17
2023-01-18
2023-01-19
2023-01-20
2023-01-21
2023-01-22
2023-01-23
2023-01-24
2023-01-25
2023-01-26
2023-01-27
2023-01-28
2023-01-29
2023-01-30
2023-01-31
2023-02-01
2023-02-02
2023-02-03
2023-02-04
2023-02-05
2023-02-06
2023-02-07
2023-02-08
2023-02-09
2023-02-10
2023-02-11
2023-02-12
2023-02-13
2023-02-14
2023-02-15
2023-02-16
2023-02-17
2023-02-18
2023-02-19
2023-02-20
2023-02-21
2023-02-22
2023-02-23
2023-02-24
2023-02-25
2023-02-26
2023-02-27
2023-02-28
2023-03-01
2023-03-02
2023-03-03
2023-03-04
2023-03-05
2023-03-06
2023-03-07
2023-03-08
2023-03-09
2023-03-10
2023-03-11
2023-03-12
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-18
2023-03-19
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-25
2023-03-26
2023-03-27
2023-03-28
2023-03-29
2023-03-30
2023-03-31
2023-04-01

In [7]:
for i in predictions_naive:
  print(i)

2023-01-01 00:00:00
2023-01-02 00:00:00
2023-01-03 00:00:00
2023-01-04 00:00:00
2023-01-05 00:00:00
2023-01-06 00:00:00
2023-01-07 00:00:00
2023-01-08 00:00:00
2023-01-09 00:00:00
2023-01-10 00:00:00
2023-01-11 00:00:00
2023-01-12 00:00:00
2023-01-13 00:00:00
2023-01-14 00:00:00
2023-01-15 00:00:00
2023-01-16 00:00:00
2023-01-17 00:00:00
2023-01-18 00:00:00
2023-01-19 00:00:00
2023-01-20 00:00:00
2023-01-21 00:00:00
2023-01-22 00:00:00
2023-01-24 00:00:00
2023-01-25 00:00:00
2023-01-26 00:00:00
2023-01-27 00:00:00
2023-01-28 00:00:00
2023-01-29 00:00:00
2023-01-30 00:00:00
2023-01-31 00:00:00
2023-02-01 00:00:00
2023-02-02 00:00:00
2023-02-03 00:00:00
2023-02-04 00:00:00
2023-02-05 00:00:00
2023-02-06 00:00:00
2023-02-07 00:00:00
2023-02-08 00:00:00
2023-02-09 00:00:00
2023-02-10 00:00:00
2023-02-11 00:00:00
2023-02-12 00:00:00
2023-02-13 00:00:00
2023-02-14 00:00:00
2023-02-15 00:00:00
2023-02-16 00:00:00
2023-02-17 00:00:00
2023-02-18 00:00:00
2023-02-19 00:00:00
2023-02-20 00:00:00
